# Experiment 5: Scattering Features + CNN Model




In [9]:
import sys
sys.path.append('../src')

from utils.reduce import reduce_pca
from utils.split import train_test_split, train_test_split_pytorch
from utils.UltrasoundDataset import UltrasoundDataset
from utils.Networks import  BasicBlock, Scattering2dResNet
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
from kymatio.torch import Scattering2D
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim
import pickle
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt

In [12]:
# create an experiment in mlruns if doesn't exit and specifiy where to log
mlflow.set_experiment('scattering_cnn_experiment')

## Upload Ultrasound images

In [10]:
with open('../data/02_interim/bmodes_steatosis_assessment_IJCARS.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [11]:
M, N= 434, 636 # ultrasound image dimension

In [13]:
# split training and test (by making sure the 10 ultrasound images of one patient is in the same set)
train_data, test_data = train_test_split(df)

In [14]:
###############################################################################
# If a GPU is available, let's use it!
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
###############################################################################

# Create dataloader

In [ ]:
batch_size

In [8]:
# create dataset
# from https://github.com/python-engineer/pytorchTutorial/blob/master/09_dataloader.py

train_dataset = UltrasoundDataset(train_data)
test_dataset  = UltrasoundDataset(test_data)

# Load whole dataset with DataLoader
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses
# !!! IF YOU GET AN ERROR DURING LOADING, SET num_workers TO 0 !!!
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=20,
                          shuffle=True,
                          num_workers=2)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=5,
                          shuffle=True,
                          num_workers=2)


In [9]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    # in_planes = in_channels
    # out_planes = out_channels
    return nn.Conv2d(in_planes, out_planes, kernel_size=10, stride=stride,
                     padding=1, bias=False)


In [10]:
with open('./train_dataset.pickle', 'wb') as handle:
    pickle.dump(train_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./test_dataset.pickle', 'wb') as handle:
    pickle.dump(test_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


## Defining scattering transformations

In [11]:
parser = argparse.ArgumentParser(description='CIFAR scattering  + hybrid examples')
parser.add_argument('--mode', type=int, default=1,help='scattering 1st or 2nd order')
parser.add_argument('--width', type=int, default=2,help='width factor for resnet')
args = parser.parse_args(args=[])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if args.mode == 1:
    scattering = Scattering2D(J=2, shape=(M, N), max_order=1)
    K = 17*1
else:
    scattering = Scattering2D(J=2, shape=(M, N))
    K = 81*1
if use_cuda:
    scattering = scattering.cuda()



## Training and Testing Functions

In [12]:
for batch_idx, (data, target) in enumerate(train_loader):
    print(scattering(data.to(device)).shape)
    break

torch.Size([20, 1, 17, 108, 159])


In [13]:
def train(model, device, train_loader, optimizer, epoch, scattering):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(scattering(data))
        loss = F.cross_entropy(output, target.type(torch.long))
        loss.backward()
        optimizer.step()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader, scattering):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(scattering(data))
            print(target, output.max(1, keepdim=True)[1])
            test_loss += F.cross_entropy(output, target.type(torch.long), reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


## Training

In [24]:
model = Scattering2dResNet(K, args.width).to(device)

In [25]:
from tqdm import tqdm

In [28]:
test(model, device, test_loader, scattering)

tensor([1, 1, 1, 0, 0], device='cuda:0', dtype=torch.uint8) tensor([[1],
        [0],
        [0],
        [0],
        [0]], device='cuda:0')
tensor([1, 1, 0, 0, 1], device='cuda:0', dtype=torch.uint8) tensor([[0],
        [0],
        [0],
        [1],
        [1]], device='cuda:0')
tensor([1, 1, 1, 0, 1], device='cuda:0', dtype=torch.uint8) tensor([[1],
        [0],
        [0],
        [0],
        [0]], device='cuda:0')
tensor([1, 1, 1, 0, 0], device='cuda:0', dtype=torch.uint8) tensor([[1],
        [1],
        [1],
        [0],
        [1]], device='cuda:0')
tensor([1, 0, 1, 1, 1], device='cuda:0', dtype=torch.uint8) tensor([[0],
        [1],
        [1],
        [1],
        [0]], device='cuda:0')
tensor([1, 1, 1, 1, 1], device='cuda:0', dtype=torch.uint8) tensor([[1],
        [1],
        [1],
        [1],
        [1]], device='cuda:0')
tensor([0, 1, 0, 0, 1], device='cuda:0', dtype=torch.uint8) tensor([[1],
        [1],
        [1],
        [1],
        [0]], device='cuda:0')

In [27]:
# Optimizer
lr = 0.1
for epoch in tqdm(range(0, 100)):
    if epoch%20==0:
      optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,
                                  weight_decay=0.0005)
      lr*=0.2
    train(model, device, train_loader, optimizer, epoch+1, scattering)

test(model, device, test_loader, scattering)




  0%|          | 0/100 [00:00<?, ?it/s]

Train Epoch: 1 [0/490 (0%)]	Loss: 0.048296
Train Epoch: 1 [100/490 (20%)]	Loss: 0.318545
Train Epoch: 1 [200/490 (40%)]	Loss: 0.239580
Train Epoch: 1 [300/490 (60%)]	Loss: 0.950013
Train Epoch: 1 [400/490 (80%)]	Loss: 0.384695



  1%|          | 1/100 [00:08<13:23,  8.12s/it]

Train Epoch: 2 [0/490 (0%)]	Loss: 0.653871
Train Epoch: 2 [100/490 (20%)]	Loss: 0.284926
Train Epoch: 2 [200/490 (40%)]	Loss: 0.304205
Train Epoch: 2 [300/490 (60%)]	Loss: 0.510675
Train Epoch: 2 [400/490 (80%)]	Loss: 0.293771



  2%|▏         | 2/100 [00:16<13:13,  8.10s/it]

...


 46%|████▌     | 46/100 [06:11<07:17,  8.10s/it]

Train Epoch: 47 [0/490 (0%)]	Loss: 0.010192
Train Epoch: 47 [100/490 (20%)]	Loss: 0.003215
Train Epoch: 47 [200/490 (40%)]	Loss: 0.005848
Train Epoch: 47 [300/490 (60%)]	Loss: 0.004483
Train Epoch: 47 [400/490 (80%)]	Loss: 0.003397



 47%|████▋     | 47/100 [06:19<07:09,  8.10s/it]

Train Epoch: 48 [0/490 (0%)]	Loss: 0.006691
Train Epoch: 48 [100/490 (20%)]	Loss: 0.003676
Train Epoch: 48 [200/490 (40%)]	Loss: 0.007161


KeyboardInterrupt: 

# Run Mlflow to see results

`!mlflow ui`

Should launch something like this:



In [ ]:
# !mlflow ui